In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail/Online_Retail.csv


In [16]:


# Dataset yükleme
##data = pd.read_csv("/kaggle/input/online-retail/Online_Retail.csv")
data = pd.read_csv('/kaggle/input/online-retail/Online_Retail.csv', encoding="ISO-8859-1")
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [25]:
data.dropna(subset=['CustomerID'], inplace=True)
data = data[data['Quantity'] > 0] # Remove returns
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate']) 
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


FactSales Table

In [32]:
fact_sales = data.copy()
fact_sales['Revenue'] = fact_sales['Quantity'] * fact_sales['UnitPrice']
fact_sales['DateID'] = fact_sales['InvoiceDate'].dt.strftime('%Y%m%d').astype(int)
fact_sales.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,DateID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,20101201
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,20101201
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201


Dimensiol Table

In [41]:
# Customer
dim_customer = data[['CustomerID','Country']].drop_duplicates().reset_index(drop=True)

# Product
dim_product = data[['StockCode','Description']].drop_duplicates().reset_index(drop=True)
dim_product.rename(columns={'StockCode':'ProductID','Description':'ProductName'}, inplace=True)

# Date
dim_date = pd.DataFrame()
dim_date['Date'] = pd.to_datetime(data['InvoiceDate'].dt.date.unique())
dim_date['DateID'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)
dim_date['Year'] = dim_date['Date'].dt.year
dim_date['Month'] = dim_date['Date'].dt.month
dim_date['Quarter'] = dim_date['Date'].dt.quarter
dim_date['Weekday'] = dim_date['Date'].dt.day_name


In [42]:
dim_customer.head()

,CustomerID,Country
0,17850.0,United Kingdom
1,13047.0,United Kingdom
2,12583.0,France
3,13748.0,United Kingdom
4,15100.0,United Kingdom


In [44]:
dim_product.head()

,ProductID,ProductName
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [45]:
dim_date.head()

,Date,DateID,Year,Month,Quarter,Weekday
0,2010-12-01,20101201,2010,12,4,<bound method PandasDelegate._add_delegate_acc...
1,2010-12-02,20101202,2010,12,4,<bound method PandasDelegate._add_delegate_acc...
2,2010-12-03,20101203,2010,12,4,<bound method PandasDelegate._add_delegate_acc...
3,2010-12-05,20101205,2010,12,4,<bound method PandasDelegate._add_delegate_acc...
4,2010-12-06,20101206,2010,12,4,<bound method PandasDelegate._add_delegate_acc...


KPI Measures

In [46]:
total_revenue = fact_sales['Revenue'].sum()
total_quantity = fact_sales['Quantity'].sum()
avg_basket = fact_sales.groupby('InvoiceNo')['Quantity'].sum().mean()
revenue_by_year = fact_sales.groupby(fact_sales['InvoiceDate'].dt.year)['Revenue'].sum()
yoy_growth = revenue_by_year.pct_change()


In [68]:
kpi_table = pd.DataFrame({
    'KPI': ['Total Revenue', 'Total Quantity', 'Average Basket Size', 'YoY Revenue Growth'],
    'Value': [total_revenue, total_quantity, avg_basket, yoy_growth]
})

kpi_table['Value'] = kpi_table['Value']
print(kpi_table)

                   KPI                                              Value
0        Total Revenue                                        8911407.904
1       Total Quantity                                            5181696
2  Average Basket Size                                         279.547691
3   YoY Revenue Growth  InvoiceDate
2010          NaN
2011    13.55996...


Plotly Dashboard

In [58]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values=list(kpi_table.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[kpi_table[col] for col in kpi_table.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()


In [59]:
import plotly.express as px

# Monthly Revenue Trend
monthly_revenue = fact_sales.groupby(fact_sales['InvoiceDate'].dt.to_period('M'))['Revenue'].sum().reset_index()
monthly_revenue['InvoiceDate'] = monthly_revenue['InvoiceDate'].dt.to_timestamp()
fig = px.line(monthly_revenue, x='InvoiceDate', y='Revenue', title='Monthly Revenue Trend')
fig.show()

# Top 10 Products
top_products = fact_sales.groupby('StockCode')['Revenue'].sum().sort_values(ascending=False).head(10).reset_index()
fig2 = px.bar(top_products, x='StockCode', y='Revenue', title='Top 10 Products by Revenue')
fig2.show()
